## Download packages

In [ ]:
# @title
import sys
import os

# Ensure package lists are up to date
!apt-get update -y

# Remove any existing python3-libtorrent installation to ensure a clean slate
!apt-get remove --purge python3-libtorrent -y

# Install libtorrent via pip after purging apt-get version
# This should prevent the 'Invalid version' error and ensure pip installs a usable module.
!pip install libtorrent

!pip install requests ffmpeg-python yt-dlp

# Add common system-wide Python module paths to sys.path
# These paths are retained as a general good practice, though pip installation should ideally handle paths.
apt_python_path_general = '/usr/lib/python3/dist-packages'
apt_python_path_specific = '/usr/lib/python3.12/dist-packages'

if apt_python_path_general not in sys.path and os.path.exists(apt_python_path_general):
    sys.path.insert(0, apt_python_path_general)
if apt_python_path_specific not in sys.path and os.path.exists(apt_python_path_specific):
    sys.path.insert(0, apt_python_path_specific)


### Download any files
#### Run the cells orderly

In [ ]:
UserName = "" # @param {type:"string"}
Password = "" # @param {type:"string"}

# @markdown

# @markdown ---

# @markdown

Download_Link = "" # @param {type:"string"}
# @markdown ### `FileName is required only for Torrents and Direct Download Links`
# @markdown ### But it's good to provide it always if the file is large
FileName = "" # @param {type:"string"}

ChunkSizeInMB = 500 # @param {type:"number"}

In [5]:
%%writefile dms_transfer.py
# @title
#!/usr/bin/env python3
import os, sys, time, hashlib, subprocess, warnings, shutil
import xml.etree.ElementTree as ET
import argparse

# =========================
# TRY IMPORTS
# =========================

try:
    import requests
except ImportError:
    requests = None

try:
    import getpass
except ImportError:
    pass

# =========================
# CONSTANTS
# =========================

DMS_BASE = "https://dms.uom.lk/remote.php/webdav/"
CHUNKS_DIR_REMOTE = "chunks/"
CHUNKS_DIR_LOCAL = "./chunks"
MANIFEST_NAME = "manifest.txt"

# =========================
# PROGRESS BAR
# =========================

def draw_progress(prefix, current, total, bar_len=40):
    if not total:
        pct = 0
    else:
        pct = max(0.0, min(1.0, current/total))
    filled = int(pct * bar_len)
    bar = "█" * filled + "░" * (bar_len - filled)
    sys.stdout.write(f"\r{prefix} [{bar}] {pct*100:5.1f}%")
    sys.stdout.flush()
    if total and current >= total:
        sys.stdout.write("\n")
        sys.stdout.flush()

# =========================
# DMS HELPERS
# =========================

def build_login(user, pwd):
    return f'"{user}:{pwd}"'

def dms_exists(login, url):
    cmd = f'curl -u {login} -s -o /dev/null -w "%{{http_code}}" -I "{url}"'
    try:
        out = subprocess.check_output(cmd, shell=True, stderr=subprocess.PIPE).decode().strip()
        return out == "200"
    except subprocess.CalledProcessError:
        return False
    except FileNotFoundError:
        warnings.warn("curl command not found. Please ensure curl is installed.")
        return False

def dms_mkcol(login, url):
    subprocess.call(f'curl -u {login} -s -X MKCOL "{url}"', shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)

def dms_upload(login, local, remote, label):
    cmd = f'curl -u {login} --progress-bar -T "{local}" "{remote}"'
    subprocess.call(cmd, shell=True) # progress-bar outputs directly to stderr, so no need for PIPE here
    print(f"[OK] Uploaded {label}")

def dms_download(login, remote, local, label):
    cmd = f'curl -u {login} --progress-bar -o "{local}" "{remote}"'
    subprocess.call(cmd, shell=True) # progress-bar outputs directly to stderr, so no need for PIPE here
    print(f"[OK] Downloaded {label}")

def dms_delete(login, url):
    subprocess.call(f'curl -u {login} -s -X DELETE "{url}"', shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)

# =========================
# STORAGE / QUOTA
# =========================

def get_vps_free():
    return shutil.disk_usage("/").free

def get_remote_size(url):
    if not requests:
        warnings.warn("Requests library not available, cannot determine remote size for HTTP links.")
        return -1
    try:
        r = requests.head(url, allow_redirects=True)
        return int(r.headers.get("Content-Length", -1))
    except requests.exceptions.RequestException as e:
        warnings.warn(f"Error getting remote size: {e}")
        return -1

def get_dms_quota(login):
    data = "'<?xml version=\"1.0\"?><propfind xmlns=\"DAV:\"><prop><quota-available-bytes/><quota-used-bytes/></prop></propfind>'"
    cmd = (
        f'curl -u {login} -s -H "Depth: 0" -H "Content-Type: application/xml" '
        f'-X PROPFIND --data {data} "{DMS_BASE}"'
    )
    try:
        xml = subprocess.check_output(cmd, shell=True, stderr=subprocess.PIPE).decode()
        tree = ET.fromstring(xml)
        ns = {"d": "DAV:"}
        used = tree.find(".//d:quota-used-bytes", ns)
        avail = tree.find(".//d:quota-available-bytes", ns)
        if used is not None and avail is not None:
            return int(used.text), int(avail.text)
        else:
            warnings.warn("Could not parse quota XML, some quota fields missing.")
            return -1, -1
    except (subprocess.CalledProcessError, ET.ParseError, AttributeError) as e:
        warnings.warn(f"Error getting DMS quota: {e}")
        return -1, -1

# =========================
# DOWNLOADERS
# =========================

def download_http(url, out_name):
    print("\nStarting HTTP download...\n")
    if not requests:
        print("❌ Requests library is not installed. Cannot download via HTTP.")
        sys.exit(1)
    try:
        r = requests.get(url, stream=True)
        r.raise_for_status()

        total = int(r.headers.get("Content-Length", 0))
        done = 0

        with open(out_name, "wb") as f:
            for chunk in r.iter_content(524288): # 0.5MB chunks for requests
                if chunk:
                    f.write(chunk)
                    done += len(chunk)
                    draw_progress("Downloading", done, total)

        return out_name
    except requests.exceptions.RequestException as e:
        print(f"❌ HTTP download failed: {e}")
        sys.exit(1)

def download_torrent(url, out_name):
    try:
        import libtorrent as lt
    except ImportError:
        print("❌ libtorrent library is not installed. Cannot download torrents.")
        sys.exit(1)

    ses = lt.session()
    params = {"save_path": "./"}
    handle = lt.add_magnet_uri(ses, url, params)

    print("Downloading metadata...")
    while not handle.has_metadata():
        time.sleep(0.5)

    info = handle.get_torrent_info()
    print("Starting torrent download...\n")

    while handle.status().state != lt.torrent_status.seeding:
        s = handle.status()
        pct = int(s.progress * 100)
        draw_progress("Torrent", pct, 100)
        time.sleep(1)

    tor_name = info.name()
    print("\nTorrent download complete.\n")

    if os.path.isdir(tor_name):
        zip_name = out_name + ".zip"
        shutil.make_archive(out_name, "zip", tor_name)
        return zip_name

    return tor_name

# =========================
# CHUNKING
# =========================

def sha256(path):
    h = hashlib.sha256()
    with open(path, "rb") as f:
        for b in iter(lambda: f.read(4096), b""):
            h.update(b)
    return h.hexdigest()

def split_file(path, chunk_size_bytes_val):
    print("\nSplitting process started.")
    os.makedirs(CHUNKS_DIR_LOCAL, exist_ok=True)
    size = os.path.getsize(path)
    base = os.path.basename(path)
    read = 0
    num = 0

    print(f"\nSplitting into chunks ({size/1e9:.2f} GB) with size {chunk_size_bytes_val/1e6:.2f} MB...\n")

    with open(path, "rb") as src:
        while True:
            chunk = src.read(chunk_size_bytes_val)
            if not chunk:
                break
            name = f"{base}.part{num:03d}"
            with open(f"{CHUNKS_DIR_LOCAL}/{name}", "wb") as out:
                out.write(chunk)
            read += len(chunk)
            draw_progress("Chunking", read, size)
            num += 1

    print(f"\nCreated {num} chunks.")
    return num

def create_manifest():
    files = sorted(os.listdir(CHUNKS_DIR_LOCAL))
    with open(MANIFEST_NAME, "w") as mf:
        for f in files:
            full = f"{CHUNKS_DIR_LOCAL}/{f}"
            mf.write(f"{sha256(full)}  {f}\n")
    return files

def load_manifest(path):
    table = {}
    with open(path, "r") as mf:
        for line in mf:
            if line.strip():
                try:
                    h, name = line.split(maxsplit=1)
                    table[name.strip()] = h.strip()
                except ValueError:
                    warnings.warn(f"Skipping malformed line in manifest: {line.strip()}")
    return table

# =========================
# PRODUCER MODE (COLAB)
# =========================

def do_producer(username, password, download_link, file_name, chunk_size_in_mb):
    print("=== PRODUCER MODE (Colab) ===")

    user = username.strip()
    pwd = password.strip()
    login = build_login(user, pwd)

    # Set CHUNK_SIZE_BYTES based on the passed argument
    chunk_size_bytes_current = int(chunk_size_in_mb * 1024 * 1024)

    used, avail = get_dms_quota(login)
    print(f"\nDMS Used : {used/1e9:.2f} GB")
    print(f"DMS Free : {avail/1e9:.2f} GB")

    free = get_vps_free()
    print(f"Colab Free Storage: {free/1e9:.2f} GB\n")

    url = download_link.strip()
    out = file_name.strip()

    remote_size = -1
    if not url.startswith("magnet:?"):
        remote_size = get_remote_size(url)
        print(f"Remote size: {remote_size/1e9:.2f} GB")

    # STORAGE CHECK
    # Need to check if Colab has space for the full download (remote_size)
    # and if DMS has space for at least one chunk.
    # Note: remote_size might be -1 if it couldn't be determined.
    if remote_size > 0 and remote_size >= free:
        print("❌ Not enough storage in Colab for the full file download.")
        sys.exit(1)

    if avail > 0 and chunk_size_bytes_current >= avail:
        print("❌ DMS does not have space for even one chunk.")
        sys.exit(1)
    elif avail == -1:
        warnings.warn("Could not determine DMS quota. Proceeding without full space check for DMS.")


    # DOWNLOAD
    if url.startswith("magnet:?"):
        path = download_torrent(url, out)
    else:
        path = download_http(url, out)

    if not os.path.exists(path):
        print(f"❌ Downloaded file '{path}' not found. Exiting.")
        sys.exit(1)

    # CHUNK + MANIFEST
    split_file(path, chunk_size_bytes_current)
    chunks = create_manifest()

    # UPLOAD
    remote_dir = DMS_BASE + CHUNKS_DIR_REMOTE
    dms_mkcol(login, remote_dir)

    print("\nUploading manifest…")
    dms_upload(login, MANIFEST_NAME, remote_dir + MANIFEST_NAME, "manifest")

    print("\nChunks uploading process started.")
    for f in chunks:
        local = f"{CHUNKS_DIR_LOCAL}/{f}"
        # Check for available space before uploading each chunk
        counter = 0
        while True:
            used, avail = get_dms_quota(login)
            if avail == -1: # Error getting quota, proceed with upload
                warnings.warn("Could not get DMS quota, proceeding with upload without space check.")
                break
            if avail >= chunk_size_bytes_current:
                print(f"\n[INFO] Enough space available ({avail/1e9:.2f} GB) for chunk {f} ({chunk_size_bytes_current/1e9:.2f} GB). Proceeding with upload.")
                break
            else:
                if (counter % 5 == 0):
                    print(f"\n[WARNING] Insufficient space ({avail/1e9:.2f} GB) for chunk {f} ({chunk_size_bytes_current/1e9:.2f} GB). Waiting for space...")
                counter += 1
                time.sleep(20) # Wait for 20 seconds before re-checking
        dms_upload(login, local, remote_dir + f, f)

    print("\n✔ All chunks uploaded.")
    print("Producer mode completed.\n")

# =========================
# CONSUMER MODE (LAPTOP)
# =========================

def do_consumer(username, password):
    print("=== CONSUMER MODE (Laptop) ===")

    user = username.strip()
    pwd = password.strip()
    login = build_login(user, pwd)

    remote_dir = DMS_BASE + CHUNKS_DIR_REMOTE
    remote_manifest = remote_dir + MANIFEST_NAME

    print("Waiting for manifest...")
    # Add a timeout for waiting for the manifest to exist
    timeout_start = time.time()
    TIMEOUT_SECONDS = 300 # 5 minutes
    while not dms_exists(login, remote_manifest):
        if time.time() - timeout_start > TIMEOUT_SECONDS:
            print(f"❌ Timeout: Manifest not found after {TIMEOUT_SECONDS} seconds.")
            sys.exit(1)
        time.sleep(3)

    dms_download(login, remote_manifest, MANIFEST_NAME, "manifest")

    manifest = load_manifest(MANIFEST_NAME)
    chunk_files = sorted(manifest.keys())

    os.makedirs(CHUNKS_DIR_LOCAL, exist_ok=True)

    total = len(chunk_files)
    done = 0

    for f in chunk_files:
        remote = remote_dir + f
        local = f"{CHUNKS_DIR_LOCAL}/{f}"
        expected = manifest.get(f) # Use .get() to avoid KeyError

        if not expected:
            warnings.warn(f"Chunk '{f}' found in remote but not in manifest. Skipping.")
            continue

        print(f"\n=== Chunk {f} ===")
        # Add a timeout for waiting for each chunk to exist
        timeout_start = time.time()
        while not dms_exists(login, remote):
            if time.time() - timeout_start > TIMEOUT_SECONDS:
                print(f"❌ Timeout: Chunk '{f}' not found after {TIMEOUT_SECONDS} seconds.")
                sys.exit(1)
            time.sleep(3)

        # retry checksum mismatch
        for attempt in range(3):
            dms_download(login, remote, local, f)
            if os.path.exists(local):
                actual = sha256(local)
                if actual == expected:
                    break
                print("Checksum mismatch, retrying…")
                os.remove(local)
            else:
                print(f"Downloaded chunk '{local}' not found, retrying…")
            time.sleep(5) # Wait before retrying download
        else:
            print("❌ Failed to verify chunk after multiple attempts.")
            sys.exit(1)

        dms_delete(login, remote)
        done += 1
        draw_progress("Total download", done, total)

    print("\nMerging chunks…")

    if not chunk_files:
        print("❌ No chunks to merge. Exiting.")
        sys.exit(1)

    first = chunk_files[0]
    base = first.split(".part")[0]
    final = base

    try:
        with open(final, "wb") as out:
            for f in chunk_files:
                local = f"{CHUNKS_DIR_LOCAL}/{f}"
                if os.path.exists(local):
                    with open(local, "rb") as src:
                        shutil.copyfileobj(src, out)
                else:
                    warnings.warn(f"Local chunk file '{local}' not found during merge. Skipping.")
        print(f"\n✔ File merged into: {final}")
    except Exception as e:
        print(f"❌ Error during merging chunks: {e}")
        sys.exit(1)


    shutil.rmtree(CHUNKS_DIR_LOCAL, ignore_errors=True)
    if os.path.exists(MANIFEST_NAME):
        os.remove(MANIFEST_NAME)

    print("\nConsumer mode completed.\n")

# =========================
# MAIN (for standalone execution)
# =========================

def main_cli():
    parser = argparse.ArgumentParser(description="DMS File Transfer Script")
    parser.add_argument("--mode", choices=["producer", "consumer"], required=True, help="Mode of operation: 'producer' for uploading, 'consumer' for downloading.")
    parser.add_argument("--username", help="DMS Username", required=True)
    parser.add_argument("--password", help="DMS Password", required=True)
    parser.add_argument("--download_link", help="Link to download (for producer mode only)")
    parser.add_argument("--file_name", help="Output file name (for producer mode only)")
    parser.add_argument("--chunk_size_in_mb", type=int, default=500, help="Chunk size in MB (for producer mode only). Default is 500 MB.")
    args = parser.parse_args()

    if args.mode == "producer":
        if not args.download_link or not args.file_name:
            parser.error("--download_link and --file_name are required for producer mode.")
        do_producer(args.username, args.password, args.download_link, args.file_name, args.chunk_size_in_mb)
    else: # consumer
        do_consumer(args.username, args.password)

if __name__ == "__main__":
    main_cli()

Overwriting dms_transfer.py


# Run the python code

In [ ]:
import dms_transfer
dms_transfer.do_producer(
    username=UserName,
    password=Password,
    download_link=Download_Link,
    file_name=FileName,
    chunk_size_in_mb=ChunkSizeInMB
)

## To upload a specific file
#### Because we can have keep the files size more the allocated quota in webdav (just delete and then restore). But it'll remove within some period.

In [ ]:
# You need run the above cell first to install the required packages

import getpass
from dms_transfer import build_login, dms_upload, DMS_BASE, CHUNKS_DIR_REMOTE, CHUNKS_DIR_LOCAL

# Define the specific chunk to upload
chunk_filename = "supernatural_S04.zip.part002"  # Replace with your desired chunk filename

# Get DMS credentials
user = UserName.strip()
pwd = Password.strip()
login = build_login(user, pwd)

# Construct local and remote paths for the chunk
local_path = f"{CHUNKS_DIR_LOCAL}/{chunk_filename}"
remote_path = DMS_BASE + CHUNKS_DIR_REMOTE + chunk_filename

# Upload the specific chunk
print(f"\nAttempting to upload: {chunk_filename}")
dms_upload(login, local_path, remote_path, chunk_filename)

print("\nUpload of specified chunk completed.")